In [1]:
from tkinter import *
from tkinter import ttk
import numpy as np
import pandas as pd
from yahoofinancials import YahooFinancials
import plotly.express as px
import scipy.optimize as opt
import pandas_datareader.data as web
import datetime as date
import mplfinance as mpf
import yfinance as yf
from scipy.stats import norm
import matplotlib.pyplot as plt
import data as dt

Real Estate


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:04<00:00,  3.31it/s]


In [2]:
tickerr = dt.Tickets[:7]

In [3]:
#tickerr = ["AAPL", "FB", "SHOP", "C", "AMZN", "MSFT", "EPD", "KO", "TSLA","WMT"]
prices = web.DataReader(name = tickerr, data_source = "yahoo",
              start= "2022-01-01")["Adj Close"].dropna()

In [4]:
resultado=np.zeros(len(tickerr))
for i in range(len(tickerr)):
    datos=yf.download(tickerr[i], period='1d', interval='1m')
    price=datos.iloc[-1,3]
    resultado[i]=price

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
prices.iloc[-1,:] = resultado
prices

Symbols,PLD,AMT,CCI,PSA,EQIX,WELL,O
Date,,,,,,,
2022-01-03,164.057190,286.380005,205.981216,363.251038,832.514160,84.940948,70.438484
2022-01-04,160.177139,282.769989,200.438538,363.002441,811.126587,87.502289,71.477272
2022-01-05,156.844254,264.570007,186.864441,359.760529,785.706360,86.350685,70.319786
2022-01-06,155.769775,264.890015,190.870239,363.310730,771.746643,86.995972,70.646255
2022-01-07,152.944290,266.579987,193.081360,353.933105,771.607239,87.194534,70.646255
...,...,...,...,...,...,...,...
2022-04-06,170.110001,268.619995,197.570007,408.579987,766.570007,97.059998,71.779999
2022-04-07,168.830002,265.730011,196.520004,409.200012,759.229980,96.290001,71.620003
2022-04-08,168.649994,266.410004,198.020004,408.950012,768.780029,96.980003,72.160004


In [6]:
#Analisis RSI 
# data = yf.download(tickers, period='1d', interval='1m')
data = prices

RSII = pd.DataFrame()
for i in range(len(tickerr)):
    
    delta = data.iloc[:,i].diff(1)
    delta.dropna(inplace=True)

    positive = delta.copy()
    negative = delta.copy()

    positive[positive < 0 ]=0
    negative[negative > 0 ]=0
    days = 14
    avrg_gain = positive.rolling(window=days).mean()
    avrg_loss = abs(negative.rolling(window=days).mean())

    rltv_strength = avrg_gain/avrg_loss
    RSI = 100-(100/(1+rltv_strength))

    combined = pd.DataFrame()
    combined["Close"] =  data.iloc[:,i]
    combined["RSI"] = RSI
    RSII[i] = RSI

    
RSII  = pd.DataFrame(RSII)
RSII.columns = tickerr
RSI_HOY = RSII.iloc[len(RSII)-1,:]

In [7]:
RSI_HOY=pd.DataFrame(RSI_HOY)
RSI_HOY

,2022-04-12
Ticker,
PLD,63.140824
AMT,74.441165
CCI,83.741405
PSA,77.375542
EQIX,61.322712
WELL,62.359180
O,75.661420


In [8]:
ws  = Tk()
ws.title('RSI Analysis')
ws.geometry('500x500')
#ws['bg'] = '#AC99F2'
game_frame = Frame(ws)
game_frame.pack()
my_game = ttk.Treeview(game_frame)
my_game['columns'] = ('TICKER','STATUS','PRICE')

my_game.column("#0", width=0,  stretch=NO)
my_game.column("TICKER",anchor=CENTER, width=80)
my_game.column("STATUS",anchor=CENTER,width=80)
my_game.column("PRICE",anchor=CENTER,width=80)

my_game.heading("#0",text="",anchor=CENTER)
my_game.heading("TICKER",text="TICKER",anchor=CENTER)
my_game.heading("STATUS",text="STATUS",anchor=CENTER)
my_game.heading("PRICE",text="PRICE",anchor=CENTER)

for i in range(len(tickerr)-1):
    if(RSI_HOY.iloc[i,0]<30):
        my_game.insert(parent='',index='end',text='', values=(tickerr[i],'BUY',np.round((resultado[i]),4)))
    if(RSI_HOY.iloc[i,0]>70):
        my_game.insert(parent='',index='end',text='', values=(tickerr[i],'SELL',np.round((resultado[i]),4)))
    if((RSI_HOY.iloc[i,0]>=30) & (RSI_HOY.iloc[i,0]<=70)):
        my_game.insert(parent='',index='end',text='', values=(tickerr[i],'HOLD',np.round((resultado[i]),4)))

my_game.pack()

ws.mainloop()